# Normative modelling for dMRI data

In [2]:
import os
os.chdir('/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/braincharts/scripts/')

import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import matplotlib.colors
import seaborn as sns
import sys
from random import sample

from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from nm_utils import calibration_descriptives, remove_bad_subjects, load_2d

ukb_root = '/project_freenas/3022017.02/UKB'
sys.path.append(os.path.join(ukb_root,'scripts'))
from ukb_utils import get_variables_UKB, lookup_UKB
ukb_idp_dir = os.path.join(ukb_root,'phenotypes','current')
root_dir = '/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/braincharts'

### Load data

In [3]:
# load sex
field_codes = ['eid','31-0.0']
field_names = ['eid', 'sex']
df_sex, subs = get_variables_UKB(os.path.join(ukb_idp_dir,'01_basic_demographics.csv'), field_codes, field_names)

# load age and site
field_codes = ['eid', '21003-2.0', '54-2.0']
field_names = ['eid', 'age', 'site']
df_age, subs = get_variables_UKB(os.path.join(ukb_idp_dir,'99_miscellaneous.csv'), field_codes, field_names)

# load sex
field_codes = ['eid','31-0.0']
field_names = ['eid', 'sex']
df_sex, subs = get_variables_UKB(os.path.join(ukb_idp_dir,'01_basic_demographics.csv'), field_codes, field_names)

# load age and site
field_codes = ['eid', '21003-2.0', '54-2.0']
field_names = ['eid', 'age', 'site']
df_age, subs = get_variables_UKB(os.path.join(ukb_idp_dir,'99_miscellaneous.csv'), field_codes, field_names)

# load dMRI derived phenotypes
field_codes = ['eid', 
               '25059-2.0', 
               '25101-2.0',
               '25100-2.0', 
               '25061-2.0',
               '25063-2.0', 
               '25062-2.0',
               '25107-2.0', 
               '25149-2.0',
               '25148-2.0', 
               '25109-2.0',
               '25111-2.0',
               '25110-2.0',
               '25347-2.0', 
               '25389-2.0',
               '25388-2.0', 
               '25349-2.0',
               '25351-2.0', 
               '25350-2.0',
               '25443-2.0', 
               '25485-2.0',
               '25484-2.0', 
               '25445-2.0',
               '25447-2.0', 
               '25446-2.0',]
field_names = ['eid', 
               'Mean_FA_in_body_of_corpus_callosum_on_FA_skeleton', 
               'Mean_FA_in_uncinate_fasciculus_on_FA_skeleton_left',
               'Mean_FA_in_uncinate_fasciculus_on_FA_skeleton_right',
               'Mean_FA_in_fornix_on_FA_skeleton',
               'Mean_FA_in_corticospinal_tract_on_FA_skeleton_left',
               'Mean_FA_in_corticospinal_tract_on_FA_skeleton_right',
               'Mean_MD_in_body_of_corpus_callosum_on_FA_skeleton', 
               'Mean_MD_in_uncinate_fasciculus_on_FA_skeleton_left',
               'Mean_MD_in_uncinate_fasciculus_on_FA_skeleton_right',
               'Mean_MD_in_fornix_on_FA_skeleton',
               'Mean_MD_in_corticospinal_tract_on_FA_skeleton_left',
               'Mean_MD_in_corticospinal_tract_on_FA_skeleton_right',
               'Mean_ICVF_in_body_of_corpus_callosum_on_FA_skeleton', 
               'Mean_ICVF_in_uncinate_fasciculus_on_FA_skeleton_left',
               'Mean_ICVF_in_uncinate_fasciculus_on_FA_skeleton_right',
               'Mean_ICVF_in_fornix_on_FA_skeleton',
               'Mean_ICVF_in_corticospinal_tract_on_FA_skeleton_left',
               'Mean_ICVF_in_corticospinal_tract_on_FA_skeleton_right',
               'Mean_ISOVF_in_body_of_corpus_callosum_on_FA_skeleton', 
               'Mean_ISOVF_in_uncinate_fasciculus_on_FA_skeleton_left',
               'Mean_ISOVF_in_uncinate_fasciculus_on_FA_skeleton_right',
               'Mean_ISOVF_in_fornix_on_FA_skeleton',
               'Mean_ISOVF_in_corticospinal_tract_on_FA_skeleton_left',
               'Mean_ISOVF_in_corticospinal_tract_on_FA_skeleton_right']
# field_codes = ['eid','25107-2.0']
# field_names = ['eid', 'Mean_MD_in_body_of_corpus_callosum_on_FA_skeleton']
df_dmri, subs = get_variables_UKB(os.path.join(ukb_idp_dir,'31_brain_IDPs.csv'), field_codes, field_names)

In [4]:
dmri_data = df_sex.join(df_age).join(df_dmri)
dmri_data.Mean_MD_in_body_of_corpus_callosum_on_FA_skeleton = dmri_data.Mean_MD_in_body_of_corpus_callosum_on_FA_skeleton*1000
dmri_data.Mean_MD_in_uncinate_fasciculus_on_FA_skeleton_left = dmri_data.Mean_MD_in_uncinate_fasciculus_on_FA_skeleton_left*1000
dmri_data.Mean_MD_in_uncinate_fasciculus_on_FA_skeleton_right = dmri_data.Mean_MD_in_uncinate_fasciculus_on_FA_skeleton_right*1000
dmri_data.Mean_MD_in_fornix_on_FA_skeleton = dmri_data.Mean_MD_in_fornix_on_FA_skeleton*1000
dmri_data.Mean_MD_in_corticospinal_tract_on_FA_skeleton_left = dmri_data.Mean_MD_in_corticospinal_tract_on_FA_skeleton_left*1000
dmri_data.Mean_MD_in_corticospinal_tract_on_FA_skeleton_right = dmri_data.Mean_MD_in_corticospinal_tract_on_FA_skeleton_right*1000
dmri_data.dropna(inplace=True)
dmri_data.to_csv(os.path.join(root_dir,'data_dmri.csv'))
dmri_data.to_pickle(os.path.join(root_dir,'data_dmri.pkl'))

### Load 500 subjects sample

In [6]:
# 500 labeled sample
script_dir = '/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm'
pickled_data = os.path.join(script_dir,'500ukb_qcfeatures.pkl')
with open(pickled_data, 'rb') as f: 
    qc500_features = pickle.load(f)
qc500_path = '/home/preclineu/ramcir/Desktop/Diffusion/qc/dMRI/QC_500.csv'
qc500_labels = pd.read_csv(qc500_path)
qc500_labels["Score"] = [0 if ele > 2 else 1 for ele in qc500_labels["Score"]] # replace 3 scores with 0 and 2's with 1's
qc500_labels = qc500_labels.fillna(1) #replace nans with 1s
# Select the covariates and IDPs for the 500 subjects with labels
df_500sample = dmri_data[dmri_data.index.isin(qc500_labels.ID.astype(str))]

### Split the data

In [7]:
### Load all the subject IDs that have qc data available
pickled_data = '/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/subjects_qc.pkl'
with open(pickled_data, 'rb') as f: 
    subs_qc = pickle.load(f)
### Load the subjects IDs that have manual qc labels available 
pickled_data = '/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/labeledsubs_qc.pkl'
with open(pickled_data, 'rb') as f: 
    labeled_qc = pickle.load(f)
### Subtract the labeled subs from the total subs
keys = list(labeled_qc.columns.values)
i1 = subs_qc.set_index(keys).index
i2 = labeled_qc.set_index(keys).index
unlabeled_subs = subs_qc[~i1.isin(i2)]
print('There are', len(subs_qc), 'subjects in total')
print('There are', len(unlabeled_subs), 'subjects without manually assigned labels')
pickled_data2 = '/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/subs_test_5k.pkl'
with open(pickled_data2, 'rb') as f: 
    subs_test = pickle.load(f)
### Create the training subs list
keys = list(subs_test.columns.values)
i1 = unlabeled_subs.set_index(keys).index
i2 = subs_test.set_index(keys).index
subs_train = unlabeled_subs[~i1.isin(i2)]
print('There are', len(subs_train), 'subjects for training')
print('There are', len(subs_test), 'subjects for testing')
# subs_train.to_pickle('/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/subs_train_5k.pkl')

There are 23158 subjects in total
There are 22658 subjects without manually assigned labels
There are 17658 subjects for training
There are 5000 subjects for testing


### Configure model parameters

In [9]:
# Create the training and testig dataset based on the subject lists
df_tr = dmri_data[dmri_data.index.isin(subs_train.subs)]
df_te = dmri_data[dmri_data.index.isin(subs_test.subs)]
# df_te = df_500sample

# with open(os.path.join(root_dir,'data','phenotypes.txt')) as f:
#     idp_ids = f.read().splitlines()
# idp_ids = ['Mean_FA_in_body_of_corpus_callosum_on_FA_skeleton', 
#            'Mean_FA_in_uncinate_fasciculus_on_FA_skeleton_left',
#            'Mean_FA_in_uncinate_fasciculus_on_FA_skeleton_right',
#            'Mean_FA_in_fornix_on_FA_skeleton',
#            'Mean_FA_in_corticospinal_tract_on_FA_skeleton_left',
#            'Mean_FA_in_corticospinal_tract_on_FA_skeleton_right',
#            'Mean_MD_in_body_of_corpus_callosum_on_FA_skeleton', 
#            'Mean_MD_in_uncinate_fasciculus_on_FA_skeleton_left',
#            'Mean_MD_in_uncinate_fasciculus_on_FA_skeleton_right',
#            'Mean_MD_in_fornix_on_FA_skeleton',
#            'Mean_MD_in_corticospinal_tract_on_FA_skeleton_left',
#            'Mean_MD_in_corticospinal_tract_on_FA_skeleton_right',
#            'Mean_ICVF_in_body_of_corpus_callosum_on_FA_skeleton', 
#            'Mean_ICVF_in_uncinate_fasciculus_on_FA_skeleton_left',
#            'Mean_ICVF_in_uncinate_fasciculus_on_FA_skeleton_right',
#            'Mean_ICVF_in_fornix_on_FA_skeleton',
#            'Mean_ICVF_in_corticospinal_tract_on_FA_skeleton_left',
#            'Mean_ICVF_in_corticospinal_tract_on_FA_skeleton_right',
#            'Mean_ISOVF_in_body_of_corpus_callosum_on_FA_skeleton', 
#            'Mean_ISOVF_in_uncinate_fasciculus_on_FA_skeleton_left',
#            'Mean_ISOVF_in_uncinate_fasciculus_on_FA_skeleton_right',
#            'Mean_ISOVF_in_fornix_on_FA_skeleton',
#            'Mean_ISOVF_in_corticospinal_tract_on_FA_skeleton_left',
#            'Mean_ISOVF_in_corticospinal_tract_on_FA_skeleton_right']
idp_ids = ['Mean_ISOVF_in_uncinate_fasciculus_on_FA_skeleton_left'] 
site_ids =  sorted(set(df_tr['site'].to_list()))

# which data columns do we wish to use as covariates? 
cols_cov = ['age','sex']

# which warping function to use? We can set this to None in order to fit a vanilla Gaussian noise model
warp =  'WarpSinArcsinh'

# limits for cubic B-spline basis 
xmin = 45 
xmax = 85

# Do we want to force the model to be refit every time? 
force_refit = True

# Absolute Z treshold above which a sample is considered to be an outlier (without fitting any model)
outlier_thresh = 100

# where the raw data are stored
data_dir = os.path.join(root_dir,'data')

# where the analysis takes place
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

### Fit the model

In [29]:
for idp_num, idp in enumerate(idp_ids): 
    print('Running IDP', idp_num, idp, ':')
   
    # set output dir 
    idp_dir = os.path.join(out_dir, idp)
    os.makedirs(os.path.join(idp_dir), exist_ok=True)
    os.chdir(idp_dir)
    
    # extract the response variables for training and test set
    y_tr = df_tr[idp].to_numpy() 
    y_te = df_te[idp].to_numpy()
    
    
    hyp0 = np.zeros(4)
    
    
    # write out the response variables for training and test
    resp_file_tr = os.path.join(idp_dir, 'resp_tr.txt')
    resp_file_te = os.path.join(idp_dir, 'resp_te.txt') 
    np.savetxt(resp_file_tr, y_tr)
    np.savetxt(resp_file_te, y_te)
        
    # configure the design matrix
    X_tr = create_design_matrix(df_tr[cols_cov], 
                                xmin = xmin, 
                                xmax = xmax)
    X_te = create_design_matrix(df_te[cols_cov], 
                                basis = 'bspline', 
                                xmin = xmin, 
                                xmax = xmax)

    # configure and save the covariates
    cov_file_tr = os.path.join(idp_dir, 'cov_bspline_tr.txt')
    cov_file_te = os.path.join(idp_dir, 'cov_bspline_te.txt')
    np.savetxt(cov_file_tr, X_tr)
    np.savetxt(cov_file_te, X_te)
    
    suffix = 'predict'
    
    print('Estimating the normative model...')
    estimate(cov_file_tr, resp_file_tr, testresp=resp_file_te, 
                 testcov=cov_file_te, alg='blr', optimizer = 'powell', l=1000,
                 savemodel=True, warp=warp, warp_reparam=True) #'l-bfgs-b'
    
    suffix = 'estimate'

    predict(cov_file_tr, 
                alg='blr', 
                respfile=resp_file_tr, 
                model_path=os.path.join(idp_dir,'Models'),
                outputsuffix='training')

Running IDP 0 Mean_ISOVF_in_uncinate_fasciculus_on_FA_skeleton_left :
Estimating the normative model...
Processing data in /home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/braincharts/models/test/Mean_ISOVF_in_uncinate_fasciculus_on_FA_skeleton_left/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters


/home/preclineu/ramcir/.conda/envs/myenv/lib/python3.8/site-packages/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=7.80896e-20): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/home/preclineu/ramcir/.conda/envs/myenv/lib/python3.8/site-packages/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=5.13446e-20): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/home/preclineu/ramcir/.conda/envs/myenv/lib/python3.8/site-packages/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.63146e-20): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)


Optimization terminated successfully.
         Current function value: -54049.938873
         Iterations: 18
         Function evaluations: 974
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...


In [30]:
# initialise dataframe we will use to store quantitative metrics 
blr_metrics = pd.DataFrame(columns = ['eid', 'NLL', 'EV', 'MSLL', 'BIC','Skew','Kurtosis'])

for idp_num, idp in enumerate(idp_ids): 
    idp_dir = os.path.join(out_dir, idp)
    
    # load the predictions and true data. We use a custom function that ensures 2d arrays
    # equivalent to: y = np.loadtxt(filename); y = y[:, np.newaxis]
    yhat_te = load_2d(os.path.join(idp_dir, 'yhat_' + suffix + '.txt'))
    s2_te = load_2d(os.path.join(idp_dir, 'ys2_' + suffix + '.txt'))
    y_te = load_2d(os.path.join(idp_dir, 'resp_te.txt'))
    
    with open(os.path.join(idp_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle) 
    
    # compute error metrics
    if warp is None:
        metrics = evaluate(y_te, yhat_te)  
        
        # compute MSLL manually as a sanity check
        y_tr_mean = np.array( [[np.mean(y_tr)]] )
        y_tr_var = np.array( [[np.var(y_tr)]] )
        MSLL = compute_MSLL(y_te, yhat_te, s2_te, y_tr_mean, y_tr_var)         
    else:
        warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1] 
        W = nm.blr.warp
        
        # warp predictions
        med_te = W.warp_predictions(np.squeeze(yhat_te), np.squeeze(s2_te), warp_param)[0]
        med_te = med_te[:, np.newaxis]
       
        # evaluation metrics
        metrics = evaluate(y_te, med_te)
        
        # compute MSLL manually
        y_te_w = W.f(y_te, warp_param)
        y_tr_w = W.f(y_tr, warp_param)
        y_tr_mean = np.array( [[np.mean(y_tr_w)]] )
        y_tr_var = np.array( [[np.var(y_tr_w)]] )
        MSLL = compute_MSLL(y_te_w, yhat_te, s2_te, y_tr_mean, y_tr_var)     
    
    Z = np.loadtxt(os.path.join(idp_dir, 'Z_' + suffix + '.txt'))
    [skew, sdskew, kurtosis, sdkurtosis, semean, sesd] = calibration_descriptives(Z)
    
    BIC = len(nm.blr.hyp) * np.log(y_tr.shape[0]) + 2 * nm.neg_log_lik
    
    blr_metrics.loc[len(blr_metrics)] = [idp, nm.neg_log_lik, metrics['EXPV'][0], 
                                         MSLL[0], BIC, skew, kurtosis]
    
display(blr_metrics)

blr_metrics.to_pickle(os.path.join(out_dir,'blr_metrics.pkl'))

,eid,NLL,EV,MSLL,BIC,Skew,Kurtosis
0,Mean_ISOVF_in_uncinate_fasciculus_on_FA_skelet...,-54049.938873,0.012615,-0.006734,-108060.762423,0.776509,0.772822


## Make predictions

In [31]:
for idp_num, idp in enumerate(idp_ids): 
    print('Running IDP', idp_num, idp, ':')
    idp_dir = os.path.join(out_dir, idp)
    os.chdir(idp_dir)
    
    # extract and save the response variables for the test set
    y_te = df_te[idp].to_numpy()
    y_tr = df_tr[idp].to_numpy()
    
    # save the variables
    resp_file_te = os.path.join(idp_dir, 'resp_te.txt') 
    np.savetxt(resp_file_te, y_te)
    
    resp_file_tr = os.path.join(idp_dir, 'resp_tr.txt') 
    np.savetxt(resp_file_tr, y_tr)
    
    yhat_te, s2_te, Z = predict(cov_file_te, 
                                    alg='blr', 
                                    respfile=resp_file_te, 
                                    model_path=os.path.join(idp_dir,'Models'))
    
    yhat_tr, s2_tr, Z = predict(cov_file_tr, 
                                    alg='blr', 
                                    respfile=resp_file_tr, 
                                    model_path=os.path.join(idp_dir,'Models'))

Running IDP 0 Mean_ISOVF_in_uncinate_fasciculus_on_FA_skeleton_left :
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...
